In [27]:
!git clone https://github.com/roneysco/Fake.br-Corpus.git

fatal: destination path 'Fake.br-Corpus' already exists and is not an empty directory.


In [28]:
!ls Fake.br-Corpus/full_texts/

fake  fake-meta-information  true  true-meta-information


In [0]:
import re
import glob
import string
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
%matplotlib inline
sns.set(color_codes=True)

FUNÇÕES

In [0]:
# Função que recebe path de pasta contendo arquivos de texto e retorna lista com conteúdo de cada um dos arquivos texto 
def txtToList(path, lista):
  """
    Cria lista de strings (lista) com arquivos txt contidos em path.
    lista deve ser uma lista vazia pré-definida;
    path deve terminar em '/'

  """
  filesList = sorted(glob.glob(path + '*'))
  length = len(filesList)
  for i in range(length):
    file = open(filesList[i], mode='r')
    text = file.read()
    file.close()
    lista.append(text)

In [0]:
# Função que Transforma uma lista de listas em um dataframe
def listToDataFrame(list_,cols_):
  lista = []
  for i in range(len(list_)):  
     lista.append(list_[i].split('\n'))
  
  df = pd.DataFrame(lista,columns=cols)

  return df

In [0]:
# Função pra testar se notícia tem depoimento
def temDepoimento(texto):
    """
    Recebe uma string e traz como resultado 1 se houver depoimento no formado
    aspas duplas + palavras/espaços intercalados + aspas duplas + vírgula +
    palavra.
    
    """
    resultado = int(bool(re.search("\".+\", \w+", texto)))
    return resultado

In [0]:
# Função para alterar todos os dados igual a "None" para vazio, assim é possível converter os dados para os tipos necessários
def removenone(df,cols):
  for col in cols:
    if(df[col].dtype != np.float64 and df[col].dtype != np.int64):
      df[col] = [x.replace('None','') for x in df[col]]
  
  return df

In [0]:
# Função para contagem de sinais de pontuação
def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text) - text.count(" ")), 3)*100

In [35]:
allfakes = []
txtToList('Fake.br-Corpus/full_texts/fake/', allfakes)

alltrue = []
txtToList('Fake.br-Corpus/full_texts/true/', alltrue)

allmetafakes = []
txtToList('Fake.br-Corpus/full_texts/fake-meta-information/', allmetafakes)

allmetatrue = []
txtToList('Fake.br-Corpus/full_texts/true-meta-information/', allmetatrue)

len(alltrue), len(allfakes), len(allmetatrue), len(allmetafakes)

(3600, 3600, 3600, 3600)

In [0]:
# Cria dataframe com coluna 'texto' com todas as fakenews + coluna 'classe' = 0
textfakenews = pd.DataFrame(allfakes)
textfakenews.columns = ['texto']
textfakenews['classe'] = 0

# Cria dataframe com coluna 'texto' c/ todas as truenews + coluna 'classe' = 1
texttruenews = pd.DataFrame(alltrue)
texttruenews.columns = ['texto']
texttruenews['classe'] = 1

In [37]:
cols = ['author' ,'link' ,'category' ,'date of publication' ,'number of tokens' ,'number of words without punctuation' ,'number of types' ,
        'number of links inside the news' ,'number of words in upper case' ,'number of verbs' ,'number of subjuntive and imperative verbs' ,
        'number of nouns' ,'number of adjectives' ,'number of adverbs' ,'number of modal verbs (mainly auxiliary verbs)' ,
        'number of singular first and second personal pronouns' ,'number of plural first personal pronouns' ,'number of pronouns' ,'pausality' ,
        'number of characters' ,'average sentence length' ,'average word length' ,'percentage of news with speeling errors' ,'emotiveness' ,'diversity']

# Cria dataframe com metadados das noticias Fake
metafakenews = listToDataFrame(allmetafakes,cols)
metafakenews.head()

,author,link,category,date of publication,number of tokens,number of words without punctuation,number of types,number of links inside the news,number of words in upper case,number of verbs,number of subjuntive and imperative verbs,number of nouns,number of adjectives,number of adverbs,number of modal verbs (mainly auxiliary verbs),number of singular first and second personal pronouns,number of plural first personal pronouns,number of pronouns,pausality,number of characters,average sentence length,average word length,percentage of news with speeling errors,emotiveness,diversity
0,mrk,https://ceticismopolitico.com/2017/11/30/katia...,politica,2017-11-30,211,185,120,0,6,30,1,46,7,13,5,2,0,26,2.0,815,14.2308,4.40541,0.0,0.263158,0.648649
1,None,https://ceticismopolitico.com/2017/11/24/dr-ra...,politica,2017-11-24,289,254,163,0,0,56,8,64,11,18,11,1,0,20,2.5,1205,18.1429,4.74409,0.00787402,0.241667,0.641732
2,None,https://afolhabrasil.com.br/politica/reinaldo-...,politica,2017-05-23,304,275,170,0,0,45,1,88,9,8,8,0,0,18,1.8125,1344,17.1875,4.88727,0.00363636,0.12782,0.618182
3,None,https://www.diariodobrasil.org/relatorio-assus...,politica,24/07/2017,639,572,316,1,14,87,7,175,39,21,14,0,0,34,2.68,3122,22.88,5.45804,0.00174825,0.229008,0.552448
4,None,https://www.diariodobrasil.org/radialista-amer...,politica,25/07/2017,128,111,82,0,1,21,0,31,6,8,1,1,0,12,0.894737,515,5.84211,4.63964,0.0,0.269231,0.738739


In [38]:
cols = ['author' ,'link' ,'category' ,'date of publication' ,'number of tokens' ,'number of words without punctuation' ,'number of types' ,
        'number of links inside the news' ,'number of words in upper case' ,'number of verbs' ,'number of subjuntive and imperative verbs' ,
        'number of nouns' ,'number of adjectives' ,'number of adverbs' ,'number of modal verbs (mainly auxiliary verbs)' ,
        'number of singular first and second personal pronouns' ,'number of plural first personal pronouns' ,'number of pronouns' ,'pausality' ,
        'number of characters' ,'average sentence length' ,'average word length' ,'percentage of news with speeling errors' ,'emotiveness' ,'diversity']

# Cria dataframe com metadados das noticias True
metatruenews = listToDataFrame(allmetatrue,cols)
metatruenews.head()

,author,link,category,date of publication,number of tokens,number of words without punctuation,number of types,number of links inside the news,number of words in upper case,number of verbs,number of subjuntive and imperative verbs,number of nouns,number of adjectives,number of adverbs,number of modal verbs (mainly auxiliary verbs),number of singular first and second personal pronouns,number of plural first personal pronouns,number of pronouns,pausality,number of characters,average sentence length,average word length,percentage of news with speeling errors,emotiveness,diversity
0,Naira Trindade,http://politica.estadao.com.br/blogs/coluna-do...,politica,13/12/2017,168,148,107,None,0,24,2,43,5,4,3,0,0,7,3.33333,761,24.6667,5.14189,0.0,0.134328,0.722973
1,Marco Rodrigo Almeida,http://www1.folha.uol.com.br/poder/2018/01/194...,politica,12/1/2018,1028,865,474,None,9,135,2,237,56,45,14,3,1,63,2.85965,4205,15.1754,4.86127,0.00115607,0.271505,0.547977
2,"Fernando Zuba , Pedro Ângelo E Renan Ramalho",https://g1.globo.com/mg/minas-gerais/noticia/s...,politica,7/12/2017,540,476,232,None,8,69,0,146,10,20,7,0,0,19,3.04762,2399,22.6667,5.03992,0.0,0.139535,0.487395
3,"Por Anderson Viegas, G1 MS",https://g1.globo.com/mato-grosso-do-sul/notici...,politica,31/10/2017 10h17,8634,7588,2199,0,52,1053,29,2170,443,278,174,0,2,384,3.36334,37381,24.3987,4.92633,0.00250395,0.223705,0.2898
4,Por BBC,https://g1.globo.com/educacao/noticia/por-que-...,politica,05/04/2017 09h08,955,823,452,0,10,96,1,262,36,50,8,0,0,33,3.14286,4239,19.5952,5.15067,0.0,0.240223,0.54921


In [0]:
# fakenews.info()
#fakenews.iloc[:,6:].columns.to_list()
# for nu in numeric_cols:
#   meancol = df_features[nu].median(skipna = True)


#   df_features[nu].fillna(meancol,inplace=True)

In [40]:
# Concatena os dois datasets
dfs = [fakenews, truenews]
allnews = pd.concat(dfs)

# Criação de nova feature temDepoimento, com 0 para False e 1 para True.
allnews['temDepoimento'] = allnews['texto'].apply(lambda x: temDepoimento(x))

# Criando nova feature, para contagem de sinais de pontuação
allnews['prop_pont'] = allnews['texto'].apply(lambda x: count_punct(x))

#Removendo colunas que não serão utilizadas
allnews.drop(columns=['date of publication','link','author','category'],inplace=True)

# Reset no index
allnews.reset_index(drop=True,inplace=True)

# Exibe primeiros registros do dataframe allnews
allnews.head(5)

NameError: ignored

In [0]:
#Convertendo dados para numericos
colsnumber = allnews.iloc[:,2:].columns.to_list()
allnews[colsnumber] = allnews[colsnumber].apply(pd.to_numeric, errors='coerce')

allnews.info()

In [0]:
# allnews.describe()
allnews['number of links inside the news'].unique()

In [0]:
fig, ax = plt.subplots(figsize=(15,10))         # Sample figsize in inches
sns.heatmap(allnews.corr(), annot=True, linewidths=.5, ax=ax)

In [0]:
# # Comparando distribuições de tamanho do texto
for col in colsnumber:
  maxcol = allnews[col].max()
  bins = np.linspace(0, maxcol, 100)
  plt.hist(allnews[allnews['classe']==1][col], bins, alpha=0.5, label='true')
  plt.hist(allnews[allnews['classe']==0][col], bins, alpha=0.5, label='fake')
  plt.legend(loc='best')
  plt.title(col)
  plt.show()

In [0]:
#X = allnews.iloc[:,2:].values
#y = allnews.iloc[:,1].values

In [0]:
#X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2,random_state=42)

In [0]:
#logit = LogisticRegression()
#logit.fit(X_train, y_train)

In [0]:
!pip install spacy
!python -m spacy download pt
import spacy
nlp = spacy.load("pt")

In [0]:
def tokenizer(texto):
    
    """
    Recebe uma string e traz como resultado duas listas:
    1) lista tokens, contendo tokens SEM remoção de pontuação, stopwords etc

    Spacy deve estar instalado: !pip install spacy
    É necessário fazer download do modelo pt do spacy: !python -m spacy download pt
    É necessário importar o spacy: import spacy
    É necessário carregar o modelo na variável nlp: nlp = spacy.load("pt") 

    """
    
    doc = nlp(texto)
    tokens = []
    for token in doc:
      tokens.append(token.text)
    return tokens

In [0]:
allnews['tokens'] = allnews['texto'].apply(lambda x: tokenizer(x))
allnews.head()

In [0]:
def pos_tagger(texto):
    
    """
    Recebe uma string e traz como resultado duas listas:
    2) lista pos_tags, contendo POS tags correspondentes 

    Spacy deve estar instalado: !pip install spacy
    É necessário fazer download do modelo pt do spacy: !python -m spacy download pt
    É necessário importar o spacy: import spacy
    É necessário carregar o modelo na variável nlp: nlp = spacy.load("pt") 

    """
    
    doc = nlp(texto)
    pos_tags = []
    for token in doc:
      pos_tags.append(spacy.explain(token.pos_))
    return pos_tags

In [0]:
allnews['pos_tags'] = allnews['texto'].apply(lambda x: pos_tagger(x))
allnews.head()

In [0]:
allnews['proportion of adjectives'] = (allnews['number of adjectives'] / allnews['number of tokens'])*100
allnews['proportion of nouns'] = (allnews['number of nouns'] / allnews['number of tokens'])*100
allnews['proportion of verbs'] = (allnews['number of verbs'] / allnews['number of tokens'])*100
allnews['proportion of adverbs'] = (allnews['number of adverbs'] / allnews['number of tokens'])*100
allnews['proportion of pronouns'] = (allnews['number of pronouns'] / allnews['number of tokens'])*100
allnews.head()